# Постановка задачи



###  *Cогласно легенде задачи*
*Мы работаем в компании, которая занимается продажей автомобилей с пробегом в Москве.*  
*Основная задача компании и её менеджеров — максимально быстро находить выгодные предложения (проще говоря, купить ниже рынка, а продать дороже рынка).*  
   
<img src="https://universegadgets.com/userfiles/165/381_0.webp"/>   
   
*Руководство компании просит нашу команду создать **модель, которая будет предсказывать стоимость автомобиля по его характеристикам**.*  
*Только вот незадача: исторически сложилось, что компания изначально не собирала данные. Есть только небольшой датасет с историей продаж за короткий период, которого для обучения модели будет явно мало. Его мы будем использовать для теста, остальное придется собрать самим.*  

Согласно условиям задачи завершающим этапом является участие в [сореновании на Kaggle](https://www.kaggle.com/c/sf-dst-car-price-prediction). Метрика используемая в соревновании - MAPE  (Mean Percentage Absolute Error) 

Задача естетсвенным образом разбивается на три этапа:
- **Этап I:Сбор и подгтовка данных:** 
- **Этап II: EDA полученных данных**
- **Этап III: Создание, настройка модели, участие в соревновании**
   
   
План работы:
- [Импорт библиотек, установка параметров, определение функций](#0)
 - [Импорт библиотек](#0_0)
 - [Установка параметров](#0_1)
 - [Определение функций](#0_2)
- [Анализ валидационного набора данных](#1):
 - EDA валидационного набора
 - отбор принимаемых к расмотрению моделью данных
- [Сбор и подгтовка данных](#2):
  - парсинг данных (https://auto.ru/moskva/)
  - привлечение сторонних данных
  - предобработка полученых данных (как парсинг, так и стороннние), приведение их к единому виду
- [EDA полученных данных](#3)
  - EDA собсвенного набора
  - EDA сторонних данных
  - Сравнение распределений в наборах данных
- [Создание, настройка модели, участие в соревновании](#4)
  - создание Baseline, эксперименты с Baseline
  - выбор моделей 
      - эксперименты с моделлями
      - создание признаков
      - отбор признаков
      - подбор гиперпараметров модели
- [Ансамблирование](#5)
   - Стекинг
   - Блендинг

# <a name="0"></a> Импорт библиотек, установка параметров, определение функций
## <a name="0_0"></a> Импорт бибилиотек

In [3]:
import pandas as pd
import numpy as np
import json
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns


from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm
from bs4 import BeautifulSoup

import warnings

## <a name="0_1"></a> Установка параметров

In [5]:
%pylab inline

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

# альтернативные пути для kaggle и локальный (НЕ ЗАБЫВАТЬ МЕНЯТЬ!)
path = './Project_7_data/'
# path = '/kaggle/input/.....'

# фиксация randomstate
RANDOM_SEED = 42

executable_path = 'C:/Users/GANSOR-PC/chromium/chromedriver.exe'
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuel_type', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id', 'vehicleTransmission',
       'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'offerprice'] 

Populating the interactive namespace from numpy and matplotlib


## <a name="0_2"></a> Определение функций

In [4]:
def describe_df(df):
    '''
    Вывод простой статистки DataFrame
    '''
    desc = df.describe(include='all',percentiles=[0.5]).drop(index=['mean','std','50%']).T 
    desc['type'] = [type(x) for x in df.iloc[0]]
    desc['NaN_prop'] = round(df.isna().sum()/len(df),3)
    desc.unique = df.nunique()
    desc.top = df.mode(axis=0).iloc[0]
    desc.top = [str(x)[:30] for x in df.mode(axis=0).iloc[0]]

    desc.freq = [df[col].value_counts().iloc[0] for col in  df.columns]
    
    return desc

# <a name="1"></a> Анализ  валидационных данных 

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо узнать какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем набор данных эти наблюдения описывающие.

In [11]:
df_test = pd.read_csv(f'{path}test.csv')

## EDA валидационного набора
### Общий обзор

In [12]:
df_test.info(verbose=False)
describe_df(df_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


,count,unique,top,freq,min,max,type,NaN_prop
bodyType,34686,24,седан,13147,NaN,NaN,<class 'str'>,0.000
brand,34686,12,BMW,4473,NaN,NaN,<class 'str'>,0.000
car_url,34686,34686,https://auto.ru/cars/used/sale,1,NaN,NaN,<class 'str'>,0.000
color,34686,16,чёрный,11010,NaN,NaN,<class 'str'>,0.000
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegan",51,NaN,NaN,<class 'float'>,0.815
description,34686,31732,Выгода до 82 000 руб. при обме,264,NaN,NaN,<class 'str'>,0.000
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,<class 'str'>,0.000
enginePower,34686,315,249 N12,1708,NaN,NaN,<class 'str'>,0.000
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,<class 'str'>,0.288
fuelType,34686,5,бензин,28601,NaN,NaN,<class 'str'>,0.000


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков:   
-бинарные (в колонках `vendor`, `ПТС`, `Руль`);   
-категориальные (например в колонках `bodyType`, `brand`, `color`, `Владельцы`);  
-количественные (например в `enginePower` или `mileage`).  

Некоторын колонки содержат словари или длинные строки, содержащие сразу несколько признаков. 

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по этим 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на обьявления о продаже авто на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru***. Воспользуемся этим же источником.

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключим их из рассмотрения.  
  
Теперь следует ознакомиться с  каждым признак/колонку более детально. Это позволит нам понять в каком виде/формате содержится информация в колонках и соответсвенно сформировать набор данных для обучения в том же виде/формате. Так же детальное ознакомлене позволит окончательно определится с необходимость включения информации из колонок в рассмотрение моделью. В случае включения данных в рассмотрение моделью, проведем EDA.

## Детальное ознакомление
### bodyType

# ====================== TEST & OTHER ==================